In [1]:
import pandas as pd
from ethtx import EthTx, EthTxConfig
from ethtx.models.decoded_model import DecodedTransaction
from typing import List

In [ ]:
txhash = '0x4c13a74b90bd9b6da0a648beebf355010b959ca714b3317461c0017b713fe354'

ethtx_config = EthTxConfig(
    mongo_connection_string="mongomock://localhost/ethtx",  ##MongoDB connection string,
    etherscan_api_key="VQXUYMB6T6I2E9SF67JPKP6YCYFW5P645A",  ##Etherscan API key,
    web3nodes={
        "mainnet": {
            "hook": "https://api.archivenode.io/58db800c-c787-41c1-96d5-ed35e126eb35",  # multiple nodes supported, separate them with comma
            "poa": True
        }
    },
    default_chain="mainnet",
    etherscan_urls={"mainnet": "https://api.etherscan.io/api", },
)

ethtx = EthTx.initialize(ethtx_config)
""" decoded_transaction: DecodedTransaction = ethtx.decoders.decode_transaction(
    '0x1594180567438ad258c98e263b412f55f62a23ebb5e9056089e9f11a513d4816')

print(decoded_transaction) """

In [ ]:
from ethtx.models.w3_model import W3Transaction, W3Block, W3Receipt, W3CallTree
web3provider = ethtx.providers.web3provider


# read raw transaction data directly from the node
w3transaction: W3Transaction = web3provider.get_transaction(txhash)
w3block: W3Block = web3provider.get_block(w3transaction.blockNumber)
w3receipt: W3Receipt = web3provider.get_receipt(w3transaction.hash.hex())
w3calls: W3CallTree = web3provider.get_calls(w3transaction.hash.hex())

print(w3transaction.dict())


In [ ]:
from ethtx.models.decoded_model import (
    DecodedTransfer,
    DecodedBalance,
    DecodedEvent, DecodedCall,
)
from ethtx.models.objects_model import Transaction, Event, Block, Call

# read the raw transaction from the node
transaction = Transaction.from_raw(
    w3transaction=w3transaction, w3receipt=w3receipt, w3calltree=w3calls
)

# get proxies used in the transaction
proxies = ethtx.decoders.get_proxies(transaction.root_call, "mainnet")

block: Block = Block.from_raw(
    w3block=web3provider.get_block(transaction.metadata.block_number),
    chain_id="mainnet",
)

# decode transaction components
abi_decoded_events: List[Event] = ethtx.decoders.abi_decoder.decode_events(
    transaction.events, block.metadata, transaction.metadata
)
abi_decoded_calls: DecodedCall = ethtx.decoders.abi_decoder.decode_calls(
    transaction.root_call, block.metadata, transaction.metadata, proxies
)
abi_decoded_transfers: List[
    DecodedTransfer
] = ethtx.decoders.abi_decoder.decode_transfers(abi_decoded_calls, abi_decoded_events)
abi_decoded_balances: List[DecodedBalance] = ethtx.decoders.abi_decoder.decode_balances(
    abi_decoded_transfers
)


In [ ]:

# decode a single event
raw_event: Event = transaction.events[0]
abi_decoded_event: DecodedEvent = ethtx.decoders.abi_decoder.decode_event(
    raw_event, block.metadata, transaction.metadata
)

# decode a single call
raw_call: Call = transaction.root_call.subcalls[0]
abi_decoded_call: DecodedCall = ethtx.decoders.abi_decoder.decode_call(
    raw_call, block.metadata, transaction.metadata, proxies
)


In [ ]:
from ethtx.models.decoded_model import DecodedTransactionMetadata

# semantically decode transaction components
decoded_metadata: DecodedTransactionMetadata = (
    ethtx.decoders.semantic_decoder.decode_metadata(
        block.metadata, transaction.metadata, "mainnet"
    )
)
decoded_events: List[DecodedEvent] = ethtx.decoders.semantic_decoder.decode_events(
    abi_decoded_events, decoded_metadata, proxies
)

decoded_calls: Call = ethtx.decoders.semantic_decoder.decode_calls(
    abi_decoded_calls, decoded_metadata, proxies
)
decoded_transfers: List[
    DecodedTransfer
] = ethtx.decoders.semantic_decoder.decode_transfers(
    abi_decoded_transfers, decoded_metadata
)
decoded_balances: List[
    DecodedBalance
] = ethtx.decoders.semantic_decoder.decode_balances(
    abi_decoded_balances, decoded_metadata
)


In [ ]:

""" # semantically decode a single event
decoded_event: DecodedEvent = ethtx.decoders.semantic_decoder.decode_event(
    abi_decoded_events[0], decoded_metadata, proxies
)
# semantically decode a single call
decoded_call: Call = ethtx.decoders.semantic_decoder.decode_call(
    abi_decoded_calls.subcalls[0], decoded_metadata, proxies
) """


In [ ]:
import sys
import json

sys.path.append("../src/")

from com.cryptobot.utils.logger import PrettyLogger

transaction_dict = transaction.dict()
abi_decoded_events_dict = [ev.dict() for ev in abi_decoded_events]
abi_decoded_calls_dict = decoded_calls.dict()
abi_decoded_transfers_dict = [t.dict() for t in abi_decoded_transfers]
abi_decoded_balances_dict = [b.dict() for b in abi_decoded_balances]

with open(f'../data/{txhash}_transaction.json', mode='a') as fp:
    json.dump(transaction_dict, fp, default=str)
    fp.close()

with open(f'../data/{txhash}_events.json', mode='a') as fp:
    json.dump(abi_decoded_events_dict, fp, default=str)
    fp.close()

with open(f'../data/{txhash}_calls.json', mode='a') as fp:
    json.dump(abi_decoded_calls_dict, fp, default=str)
    fp.close()

with open(f'../data/{txhash}_transfers.json', mode='a') as fp:
    json.dump(abi_decoded_transfers_dict, fp, default=str)
    fp.close()

with open(f'../data/{txhash}_balances.json', mode='a') as fp:
    json.dump(abi_decoded_balances_dict, fp, default=str)
    fp.close()


In [69]:
from com.cryptobot.utils.path import get_data_path
import sys
import json

sys.path.append("../src/")


tokens = pd.read_csv(get_data_path() + 'tokens.csv')
tokens_holders_df = pd.read_csv(get_data_path() + 'tokens_holders.csv')

# sort them by symbol
tokens.sort_values(by=['symbol'], inplace=True)
tokens_holders_df.sort_values(by=['token_symbol'], inplace=True)

whale = tokens_holders_df.copy()[tokens_holders_df['address']
                                 == '0xf977814e90da44bfa03b6295a0616a897441acec']
whale_tokens = tokens.copy()[tokens['symbol'].isin(list(whale['token_symbol']))]
whale.reset_index(drop=True, inplace=True)
whale_tokens.reset_index(drop=True, inplace=True)

whale.loc[:, ('token_holdings_usd')] = whale.qty.mul(whale_tokens.price_usd)

whale.sort_values(by=['token_holdings_usd'], ascending=False)


,index,rank,address,qty,percentage,token_address,token_symbol,token_holdings_usd
7,750,1,0xf977814e90da44bfa03b6295a0616a897441acec,1.275206e+10,3.154499e+13,0x4fabb145d64652a948d72533023f6e7a623c7c53,BUSD,1.274955e+10
41,1,2,0xf977814e90da44bfa03b6295a0616a897441acec,1.700000e+09,4.205300e+00,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,1.699930e+09
36,1252,3,0xf977814e90da44bfa03b6295a0616a897441acec,3.979036e+13,9.843007e+16,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,SHIB,4.058616e+08
40,260,11,0xf977814e90da44bfa03b6295a0616a897441acec,3.560108e+08,8.807000e-01,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,3.560108e+08
24,3408,1,0xf977814e90da44bfa03b6295a0616a897441acec,3.712397e+07,9.183418e+10,0x514910771af9ca656af840dff83e8264ecf986ca,LINK,2.665501e+08
27,1757,8,0xf977814e90da44bfa03b6295a0616a897441acec,2.258204e+08,5.586157e+11,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,MATIC,1.776816e+08
9,5909,2,0xf977814e90da44bfa03b6295a0616a897441acec,9.248948e+08,2.287928e+12,0x3506424f91fd33084466f402d5d97f05f8e3b4af,CHZ,1.767233e+08
35,5411,4,0xf977814e90da44bfa03b6295a0616a897441acec,2.136114e+08,5.284142e+11,0x3845badade8e6dff049820680d1f14bd3903a5d0,SAND,1.667382e+08
33,4410,3,0xf977814e90da44bfa03b6295a0616a897441acec,7.933300e+05,1.962474e+09,0x4a220e6096b25eadb88358cb44068a3248254675,QNT,1.228313e+08
26,5660,3,0xf977814e90da44bfa03b6295a0616a897441acec,1.717618e+08,4.248899e+11,0x0f5d2fb29fb7d3cfee444a200298f468908cc942,MANA,1.116149e+08
